In [ ]:
# Some MQTT based main files for the esp8266 sensors in DoESLiverpool
# For cheaper deployment

#      RST          |  GPIO1   TX
#       A0          |  GPIO3   RX
#       D0  GPIO16  |  GPIO5   D1  SCL
# SCK   D5  GPIO14  |  GPIO4   D2  SDA
# MISO  D6  GPIO12  |  GPIO0   D3
# MOSI  D7  GPIO13  |  GPIO2   D4  LED
# SS    D8  GPIO15  |          GND
#      3V3          |          5V

# (not able to get the mDNS to work for esp8266)

In [82]:
%serialconnect

serial exception on close write failed: [Errno 5] Input/output error
Connecting to --port=/dev/ttyUSB0 --baud=115200 
Ready.


In [ ]:
# Code below is to run just the SI7021 humidity temp sensor
# (and then the dallas temperature string)

In [29]:
%sendtofile config.txt

wifiname     DoESLiverpool
wifipassword decafbad00
mqttbroker   10.0.30.130
pinled       2
boardname    esp8266A

Sent 5 lines (112 bytes) to config.txt.


In [ ]:
# we can move some of these state functions into another file

In [73]:
%sendtofile main.py

import time, network, machine, ubinascii

fconfig = dict(x.split()  for x in open("config.txt"))

mqttipnumber = ""  # not able to make mDNS to work
pled = machine.Pin(int(fconfig["pinled"]), machine.Pin.OUT)
plight = machine.ADC(0)

si = network.WLAN()
si.active(True)
si.connect(fconfig["wifiname"], fconfig["wifipassword"])
while not si.isconnected():
    time.sleep_ms(500)
ipnumber = si.ifconfig()[0]
print(ipnumber)
for i in range(10):
    pled.value(i%2)
    time.sleep_ms(100)

from umqtt.robust import MQTTClient
client = MQTTClient(fconfig["boardname"], fconfig["mqttbroker"])
client.connect()
client.publish(fconfig["boardname"]+"/ip", ipnumber, retain=True, qos=1)
client.publish(fconfig["boardname"]+"/mac", ubinascii.hexlify(si.config('mac'),':').decode().upper(), retain=True, qos=1)

for i in range(10):
    pled.value(i%2)
    time.sleep_ms(50)

# dallas string sensor code; G=Orange, D8=Yellow, 3V3=Blue
import onewire, ds18x20
pindallas = machine.Pin(13, machine.Pin.OUT, machine.Pin.PULL_UP)
dallasobj = 5 # tries
dallasscanned = []
def dallasinitconvert():
    global dallasobj, dallasscanned
    if isinstance(dallasobj, int):
        try:
            dallasobj = ds18x20.DS18X20(onewire.OneWire(pindallas))
        except onewire.OneWireError as e:
            dallasobj -= 1
            return []
        dallasscanned = dallasobj.scan()
        print("dallasscanned", dallasscanned)
        if dallasscanned:
            sdallasqorder = {347:0, 376:1, 448:2, 475:3, 576:4, 311:5, 406:6, 157:7}
            dallasscanned.sort(key=lambda X: sdallasqorder.get(sum(X), 0))
            print("DS18B20 n=%d"%len(dallasscanned))
    if dallasscanned:
        dallasobj.convert_temp()

# SI7021 humidity sensor code
i2c = machine.I2C(scl=machine.Pin(4), sda=machine.Pin(5))
bsi7021init = False
def SI7021humiditytemp():
    global bsi7021init
    if not bsi7021init:
        i2c.writeto(0x40, b'\xFE')
        time.sleep_ms(20)
        bsi7021init = True
    i2c.writeto(0x40, b'\xE5')
    time.sleep_ms(20)   # give it time to take a reading or it fails
    bh = i2c.readfrom(0x40, 2)
    bt = i2c.readfrom_mem(0x40, 0xE0, 2)
    rh = (bh[0]<<8)+(bh[1]&0xFC)
    rt = (bt[0]<<8)+(bt[1]&0xFC)
    return ((125.0*rh)/65536)-6, ((175.25*rt)/65536)-46.85 

topichumid = fconfig["boardname"]+"/SI7021/humid"
topictemp = fconfig["boardname"]+"/SI7021/temp"
topicldr = fconfig["boardname"]+"/LDR"
topicds = [ fconfig["boardname"]+"/DS18B20/"+str(i)  for i in range(len(dallasscanned)) ]

tdallasconverttime = 0
t0 = time.ticks_ms()
plightv0 = 0
while True:
    t = time.ticks_ms()
    
    plightv = plight.read()
    if abs(plightv - plightv0) > 45:
        client.publish(topicldr, "%d"%plightv)
        plightv0 = plightv
    if dallasobj and dallasscanned and tdallasconverttime and t > tdallasconverttime + 800:
        ds = [ dallasobj.read_temp(d)  for d in dallasscanned ]
        print(ds)
        for i, t in enumerate(ds):
            client.publish(topicds[i], "%f"%t)
    
    if t < t0 + 1000:
        time.sleep_ms(10)
        continue
        
    pled.value(0)
    try:
        humid, temp = SI7021humiditytemp()
        print(humid, temp)
        client.publish(topichumid, "%f"%humid)
        client.publish(topictemp, "%f"%temp)
    except OSError as e:
        print(e)
    pled.value(1)
    
    if dallasobj and dallasscanned and tdallasconverttime and t > tdallasconverttime + 800:
        dallasinitconvert()
        tdallasconverttime = t
    t0 = t
    

Sent 107 lines (3358 bytes) to main.py.


In [72]:
# Electricity to flashes esp8266 version
%serialconnect



***Connection broken [Input/output error]
You may need to reconnect
Closing serial Serial<id=0x7fe877b62438, open=True>(port='/dev/ttyUSB2', baudrate=115200, bytesize=8, parity='N', stopbits=1, timeout=0.5, xonxoff=False, rtscts=False, dsrdtr=False)
Connecting to --port=/dev/ttyUSB0 --baud=115200 
Ready.


In [40]:
# esp32 flashing light signal
%sendtofile main.py
import machine, time
pled = machine.Pin(2, machine.Pin.OUT)
pwm = machine.PWM(pled, freq=1, duty=25)
print(pwm)


Sent 4 lines (112 bytes) to main.py.


In [74]:
%serialconnect

serial exception on close write failed: [Errno 5] Input/output error
Connecting to --port=/dev/ttyUSB2 --baud=115200 
Ready.


In [44]:
%sendtofile main.py

import time, network, machine

boardname = "esp8266C"
mqttipnumber = "10.0.30.130"  # not able to make mDNS to work
pled = machine.Pin(2, machine.Pin.OUT)
pdetector = machine.Pin(13, machine.Pin.IN)

si = network.WLAN()
si.active(True)
si.connect("DoESLiverpool", "decafbad00")
while not si.isconnected():
    time.sleep_ms(500)
ipnumber = si.ifconfig()[0]
print(ipnumber)
for i in range(10):
    pled.value(i%2)
    time.sleep_ms(100)

from umqtt.robust import MQTTClient
client = MQTTClient(boardname, mqttipnumber)
client.connect()
client.publish(boardname+"/ip", ipnumber, retain=True, qos=1)

for i in range(10):
    pled.value(i%2)
    time.sleep_ms(50)

topicname = boardname+"/ticks"
n = 0
t0 = 0
while True:
    while pdetector.value():       pass
    td = time.ticks_ms()
    while pdetector.value() == 0:  pass
    t = time.ticks_ms()
    x = (t - td)  # originally machine.time_pulse_us(pdetector, 0, 10000000)//1000
    dt = t - t0
    print(dt, x, n)
    if not (20 < x < 35):
        continue
    if dt < 15:
        continue
    try:
        client.publish(topicname, "%d %d %d" % (dt, x, n))
    except Exception as e:
        print(e)
    t0 = t
    n += 1
    pled.value(n%2) 


Sent 49 lines (1196 bytes) to main.py.


In [50]:
%serialconnect

serial exception on close write failed: [Errno 5] Input/output error
Connecting to --port=/dev/ttyUSB0 --baud=115200 
Ready.


In [23]:
%serialconnect

Connecting to --port=/dev/ttyUSB1 --baud=115200 
Ready.


In [25]:
import machine
pdetector = machine.Pin(13, machine.Pin.IN)
def callback(x):
    print(x)
pdetector.irq(trigger=machine.Pin.IRQ_FALLING, handler=callback)

In [26]:
%disconnect

attempt to exit paste mode
[\r\x03\x02] b'Pin(13)\r\nPin(13)\r\nPin(13)\r\nPin(13)\r\nPin(13)\r\nPin(13)\r\n\r\nMicroPython v1.11-8-g48dcbbe60 on 2019-05-29; ESP module with ESP8266\r\nType "help()" for more information.\r\n>>> '
Closing serial Serial<id=0x7faca5f8c390, open=True>(port='/dev/ttyUSB1', baudrate=115200, bytesize=8, parity='N', stopbits=1, timeout=0.5, xonxoff=False, rtscts=False, dsrdtr=False)
